In [1]:
import numpy

In [2]:
import sklearn

In [3]:
import survwrap

In [4]:
X, y = survwrap.load_test_data()
X.shape, y.shape

((198, 84), (198,))

### Generate a (stratified) train-test split and Scale the features (only) 

First do the stratified splitting THEN do scaling, parameterized on X_train set ONLY 

In [5]:
from sklearn.preprocessing import StandardScaler, RobustScaler

In [6]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y,
                                                                            stratify=survwrap.get_indicator(y), 
                                                                           random_state=2308)

In [7]:
scaler = StandardScaler().fit(X_train)
[X_train, X_test] = [ scaler.transform(_) for _ in  [X_train, X_test] ]
X_train.shape, X_test.shape

((148, 84), (50, 84))

In [8]:
#X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y)

balanced partitioning OK. Robst scaler damages the performance of DSM A LOT.
maybe did something wrong. It is standard scaler for now.

In [9]:
survwrap.get_indicator(y).sum(), survwrap.get_indicator(y_train).sum(), survwrap.get_indicator(y_test).sum(),


(51, 38, 13)

## check possible dimensionality reduction

In [10]:
from sklearn.decomposition import PCA

In [11]:
pca= PCA(n_components=0.995, random_state=2308).fit(X_train)
pca.n_components_

74

Only a modest dimensionality reduction is possible using PCA

In [12]:
## Stratified CV spliter for survival analysis

In [13]:
from sklearn.model_selection import RepeatedKFold, RepeatedStratifiedKFold

In [14]:
testkf= RepeatedStratifiedKFold(n_splits=5,n_repeats=2,random_state=2307)
for trn,tst in testkf.split(X_train, survwrap.get_indicator(y_train)):
    print(trn,tst) 

[  1   2   3   6   7   8   9  10  11  13  14  16  17  19  20  22  24  25
  26  28  29  30  31  32  33  34  35  36  37  39  41  42  43  45  46  47
  48  49  51  52  53  54  55  57  59  60  61  62  63  67  68  70  71  72
  74  76  77  78  79  80  82  83  84  85  86  87  88  89  90  91  92  93
  95  96  97  98  99 101 102 104 105 106 108 109 110 111 112 114 115 116
 117 118 119 120 121 122 123 125 126 127 128 129 130 131 132 133 135 136
 137 138 139 140 141 142 144 145 146 147] [  0   4   5  12  15  18  21  23  27  38  40  44  50  56  58  64  65  66
  69  73  75  81  94 100 103 107 113 124 134 143]
[  0   1   3   4   5   7   8  10  11  12  13  15  16  17  18  19  20  21
  22  23  24  25  26  27  28  29  30  31  32  33  34  35  38  39  40  42
  43  44  45  46  47  49  50  51  54  55  56  57  58  61  63  64  65  66
  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83  84
  85  87  88  89  90  91  94  95  97  98 100 101 103 104 106 107 109 110
 112 113 114 115 116 117 119 120

# test coxnet

In [15]:
coxnet = survwrap.CoxNet(rng_seed=2308)
coxnet.fit(X_train, y_train)

CoxNet(rng_seed=2308, l1_ratio=0.5, verbose=False, fit_baseline_model=False)

In [16]:
coxnet.score(X_test, y_test)

0.5330578512396694

In [17]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [18]:
def optimize(estimator, X, y, mode='sklearn-grid', user_grid=[] , cv=None):
    if mode == 'sklearn-grid':
        if not user_grid: 
            user_grid = estimator.get_parameter_grid()
        gs = GridSearchCV(estimator, user_grid, refit=True, cv=cv, n_jobs=3)
        gs.fit(X, y)
        return gs.best_estimator_, gs.best_params_, gs
    raise ValueError(f'unknown mode parameter: "{mode}"')

In [19]:
survwrap.CoxNet().get_parameter_grid()

{'l1_ratio': [0.01, 0.1, 0.25, 0.5, 0.75, 0.9, 0.99]}

In [20]:
opt_coxnet, opt_coxnet_params, opt_coxnet_search = optimize(survwrap.CoxNet(), X_train, y_train, user_grid={'l1_ratio':[0.01,0.25,0.5,0.75,0.8,0.99]},
                                                           cv=RepeatedStratifiedKFold(n_splits=5, n_repeats=2, random_state=2308).split(X_train,survwrap.get_indicator(y_train)))
opt_coxnet.score(X_test, y_test), opt_coxnet_params

(0.5764462809917356, {'l1_ratio': 0.01})

In [21]:
def test_model(model_constructor, **model_params):
    m = model_constructor(**model_params, rng_seed=2308)
    m.fit(X_train, y_train)
    o = m.score(X_train, y_train)
    s = m.score(X_test, y_test)
    print(f'Train: {o: .4f} , Test: {s:.4f}')
    return s

# Test DSM

In [22]:
grid =survwrap.DeepSurvivalMachines.get_parameter_grid(max_width=84)
grid

{'n_distr': [1, 2, 3],
 'distr_kind': ['Weibull'],
 'batch_size': [16, 32],
 'layer_sizes': [[3],
  [4],
  [5],
  [7],
  [9],
  [11],
  [14],
  [19],
  [24],
  [32],
  [41],
  [54],
  [70],
  [3, 3],
  [4, 4],
  [5, 5],
  [7, 7],
  [9, 9],
  [11, 11],
  [14, 14],
  [19, 19],
  [24, 24],
  [32, 32],
  [41, 41],
  [54, 54],
  [70, 70],
  [3, 3, 3],
  [4, 4, 4],
  [5, 5, 5],
  [7, 7, 7],
  [9, 9, 9],
  [11, 11, 11],
  [14, 14, 14],
  [19, 19, 19],
  [24, 24, 24],
  [32, 32, 32],
  [41, 41, 41],
  [54, 54, 54],
  [70, 70, 70]],
 'learning_rate': [0.005, 0.001],
 'validation_size': [0.1],
 'max_epochs': [100],
 'elbo': [False]}

In [23]:
test_model(survwrap.DeepSurvivalMachines, batch_size=16, layer_sizes=[16], n_distr=2, max_epochs=200)

  3%|███▉                                                                                                                              | 6/200 [00:00<00:06, 28.81it/s]

Train:  0.8134 , Test: 0.6178


0.6177685950413223

In [ ]:
# Stratified CV
opt_dsm, opt_dsm_params, opt_dsm_search = survwrap.optimize(survwrap.DeepSurvivalMachines(rng_seed=2309),  X_train, y_train, 
                                                   user_grid=grid,n_jobs=3)
opt_dsm.score(X_test, y_test), opt_dsm_params

 28%|██▊       | 2805/10000 [00:02<00:06, 1136.72it/s]/usr/local/ivan/Unito/conda/envs/tosa/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/ivan/Unito/conda/envs/tosa/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/ivan/Unito/conda/envs/tosa/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 444, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "/usr/local/ivan/Unito/survwrap/survwrap/auton_adapters.py", line 95, in score
    return self.harrell_score(y, self.predict(X))[0]
  File "/usr/local/ivan/Unito/survwrap/survwrap/auton_adapters.py", line 83, in harrell_score
    return concordance_index_censored(
  File "/usr/local/ivan/Unito/conda/envs/tosa

In [ ]:
survwrap.get_top_models(opt_dsm_search)

In [142]:
# Non stratified CV
#opt_dsm, opt_dsm_params, opt_dsm_search = optimize(survwrap.DeepSurvivalMachines(rng_seed=2308),  X_train, y_train, 
#                                                   user_grid=grid,cv=RepeatedKFold(n_splits=5, n_repeats=3, random_state=2308))
#opt_dsm.score(X_test, y_test), opt_dsm_params

In [41]:
cv_res = opt_dsm_search.cv_results_
rez=sorted(zip(cv_res['rank_test_score'], cv_res['mean_test_score'] , cv_res['std_test_score'], 
               cv_res['mean_test_score'] - cv_res['std_test_score'], opt_dsm_search.cv_results_['params']))
rez[:10]

[(1,
  0.6800064902521725,
  0.08640148207431089,
  0.5936050081778617,
  {'batch_size': 16,
   'distr_kind': 'Weibull',
   'elbo': False,
   'layer_sizes': [13],
   'learning_rate': 0.001,
   'max_epochs': 200,
   'n_distr': 1,
   'validation_size': 0.1}),
 (2,
  0.6778373825259126,
  0.09205344136092578,
  0.5857839411649869,
  {'batch_size': 16,
   'distr_kind': 'Weibull',
   'elbo': False,
   'layer_sizes': [17],
   'learning_rate': 0.001,
   'max_epochs': 200,
   'n_distr': 2,
   'validation_size': 0.1}),
 (3,
  0.674360686971627,
  0.05712700290002999,
  0.617233684071597,
  {'batch_size': 16,
   'distr_kind': 'Weibull',
   'elbo': False,
   'layer_sizes': [37],
   'learning_rate': 0.001,
   'max_epochs': 200,
   'n_distr': 1,
   'validation_size': 0.1}),
 (4,
  0.6707957096968238,
  0.0607100595554704,
  0.6100856501413534,
  {'batch_size': 16,
   'distr_kind': 'Weibull',
   'elbo': False,
   'layer_sizes': [29],
   'learning_rate': 0.001,
   'max_epochs': 200,
   'n_distr': 2,


In [42]:
low_bound = sorted(rez[:10], key= lambda k: k[3], reverse=True)
for lb in [ _ for _ in low_bound if _[3] > 0.5] :
    print(lb[0:4])
    test_model(survwrap.DeepSurvivalMachines, **lb[4])

(3, 0.674360686971627, 0.05712700290002999, 0.617233684071597)


  3%|███▉                                                                                                                              | 6/200 [00:00<00:07, 24.69it/s]


Train:  0.8486 , Test: 0.6653
(6, 0.6683175258595094, 0.057869604945826984, 0.6104479209136825)


  5%|██████▍                                                                                                                          | 10/200 [00:00<00:05, 35.92it/s]


Train:  0.7811 , Test: 0.6198
(7, 0.6652983137465113, 0.05494023873549321, 0.610358075011018)


...

Train:  0.8032 , Test: 0.5992
(9, 0.6597070588850419, 0.08400503850693543, 0.5757020203781065)


  2%|█▉                                                                                                                                | 3/200 [00:00<00:11, 17.28it/s]


Train:  0.6866 , Test: 0.6343
(8, 0.6598382074175131, 0.09760034612352134, 0.5622378612939918)


  2%|██▌                                                                                                                               | 4/200 [00:00<00:09, 20.63it/s]


Train:  0.8286 , Test: 0.6550


In [43]:
test_model(survwrap.DeepSurvivalMachines, batch_size=16, layer_sizes=[13], n_distr=1, max_epochs=200)

  4%|█████▏                                                                                                                            | 8/200 [00:00<00:05, 36.70it/s]

Train:  0.8299 , Test: 0.6839


0.6838842975206612

# Appunti di varie prove DSM

## Insieme di prova (Breast cancer)

Piccolo insieme bastardo. 200 elementi, 84 features e solo il 25% di eventi non censurati.
Coxnet di default va male (c-index 53%), DSM richiede cura per inziare a generalizzare.

## Stabilità risultato

1. stratificare, stratificare rispetto all'evento. Usando insiemi stratificati rispetto all'evento DSM inizia a funzionare in maniera meno erratica.
2. Se si stratifica anche la CV, L'ottimizzazione DSM tende a fornire risultati buoni per tutta le soluzioni top e anche CoxNet ci guadagna in ottimizzazione (c-index: 57%).
   potrebbe sembrare ovvio a posteriori, dato che stiamo imponendo un "contenuto di informazione" simile tra train e test. ma non mi pare una forzatura clamorosa.

# Ottimizzazione parametri DSM

1. Quello più difficile da gestire è ovviamente la topologia della rete. Usando un solo strato nascosto si vede che anche cambiando di un solo valore la dimensione si può passare da una rete che generalizza a una che fa schifo, quindi bisognerebbe fare uno scan senza salti. O fare una random search.
   Alla fine mi sono inventato un protocollo scemo ma che sembra andare ragionevolmente (su breast cancer). Fare uno scan per i valori de numeri primi compresi tra 3 e n_features/2 (olè!)
2. Reti a "parametri equivalenti", (la procedura del quant): fatti tanti test, c'è un fenomeno curioso: le reti con numero pari di strarti nascosti generalizzano male. Mah!
   Quindi alla fine ho testato sistematicamente solo quelle con strati dispari (1,3,5). Ma per quelle con 5 iniziano ad apparire problemi di stabilità numerica, probabilmente quando sono troppo grandi.
3. numero di funzioni di base (Weibull): il default è 2, ma si ottengono parecchi buoni risultati anche con una sola. Molte soluzioni con 3 non generalizzano. quindi 1 o 2.
4. Come detto sopra, la stratificazione aiuta molto, così come il fare almeno 10 split (n_splits=5, n_repeats=2) per aver una varianza abbastanza stabile. Sono riuscito a fare griglie con cv stratificate con un trucchino sklearn che implementerei
5. batch size: 10% dell'insieme di train. Sembra OK. mai ottimizzata. così come gli altri parametri legati al learning. Non mi aspetto che incidano.
6. Alla fine della fiera si possono ottenere risulati stabilmente decorosi usando 1 o 3 strati nascosto e 1 sola weibull (test c-index: 65%). Introducendo anche la doppia weibull la griglia diventa più onerosa e possono aumentare le soluzioni solo apparentemente buone ma generalizzano peggio (anche se non disastrose: 58%). Non andare oltre le 2 weibull e 3 strati.
